# Overlord AI
For information about this project, see [this README.md](README.md)

# CODE STARTS HERE

First we import all the libraries and whatnot

In [1]:
import pandas as pd
import glob

Definitions go here

In [2]:
dataPath = "../data/*.csv"
dfList = ["asdf"] # define a junk list
dfList.clear() # clear said junk list 


Read the data, using glob to get all the files at once

In [3]:
dataList = glob.glob(dataPath)
for fileName in dataList:
    dfList.append(pd.read_csv(fileName , index_col=0))
    print(len(dfList))
    print(fileName)


1
../data/ds_salaries.csv


In [4]:
for df in dfList:
    df.describe()


In [5]:
    print(df)

     work_year experience_level employment_type                   job_title  \
0         2020               MI              FT              Data Scientist   
1         2020               SE              FT  Machine Learning Scientist   
2         2020               SE              FT           Big Data Engineer   
3         2020               MI              FT        Product Data Analyst   
4         2020               SE              FT   Machine Learning Engineer   
..         ...              ...             ...                         ...   
602       2022               SE              FT               Data Engineer   
603       2022               SE              FT               Data Engineer   
604       2022               SE              FT                Data Analyst   
605       2022               SE              FT                Data Analyst   
606       2022               MI              FT                AI Scientist   

     salary salary_currency  salary_in_usd employee